<a href="https://colab.research.google.com/github/Baah134/Baah134/blob/main/Whisper/Whisper_Transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
from datasets import load_dataset

!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg


!pip install jiwer
from jiwer import wer
import whisper
import numpy as np


In [ ]:

import soundfile as sf
import pandas as pd
from tqdm import tqdm

from jiwer import wer  # Library for Word Error Rate calculation

# Load Whisper model
model = whisper.load_model("medium")

# Load Afrispeech dataset (e.g., Isizulu)
dataset = load_dataset("tobiolatunji/afrispeech-200", "isizulu", split="train", streaming=True)

# Initialize list to store results
results = []

# Process 100 samples
for idx, sample in tqdm(enumerate(dataset.take(100)), total=100, desc="Processing Samples"):
    # Extract waveform and sample rate
    audio_waveform = sample["audio"]["array"]
    sr = sample["audio"]["sampling_rate"]

    # Save the waveform as a temporary WAV file
    wav_path = f"temp_audio_{idx}.wav"
    sf.write(wav_path, audio_waveform, sr)

    # Transcribe using Whisper
    result = model.transcribe(wav_path)
    whisper_text = result["text"].strip()

    # Get actual ground truth text
    actual_text = sample["transcript"].strip()

    # Compute Word Error Rate (WER)
    error_rate = wer(actual_text, whisper_text)

    # Store results in a dictionary
    results.append({
        "Whisper Transcription": whisper_text,
        "Actual Label": actual_text,
        "Word Error Rate": round(error_rate, 4)  # Rounded to 4 decimal places
    })

    # Print progress
    print(f"\nSample {idx+1}:")
    print("Whisper:", whisper_text)
    print("Actual :", actual_text)
    print("WER    :", round(error_rate, 4))
    print("-" * 50)

# Save results to a CSV file
df = pd.DataFrame(results)
csv_filename = "whisper_transcriptions.csv"
df.to_csv(csv_filename, index=False)

print(f"\nProcessing complete! Results saved in {csv_filename}")
